In [3]:
import yaml
from pathlib import Path
import cv2
from matplotlib import pyplot as plt
import json
import math
import sys
import os

In [4]:
conf = yaml.safe_load(Path('./train.yaml').read_text())
conf2 = yaml.safe_load(Path('./test.yaml').read_text())
threhold = 128
img_id = 0
width_expanson = 0.8
height_expension = 1.
os.mkdir('./data/train_data')
def image_box(x1,x2,y1,y2,x_expansion,y_expansion):
    dis_x = x2-x1
    dis_y = y2-y1
    return max(0,int(x1-x_expansion*dis_x)),min(1279,int(x2+x_expansion*dis_x)),max(0,int(y1-y_expansion*dis_y)),min(1279,int(y2+y_expansion*dis_y))


for i in conf:
    if i['boxes'] != []:
        img = cv2.imread(i['path'],1)
        for data in i['boxes']:
            if data['occluded'] == False:
                x_max=min(data['x_max'],1279)
                x_min=max(data['x_min'],0)
                y_max=min(data['y_max'],719)
                y_min=max(data['y_min'],0)
                x_min,x_max,y_min,y_max= image_box(x_min,x_max,y_min,y_max,width_expanson,height_expension)
            
                if x_max-x_min >= threhold or y_max-y_min >= threhold:
                    
                    img_cut = cv2.resize(img[y_min:y_max,x_min:x_max,:],(64,64),interpolation = cv2.INTER_LINEAR)

                    filename = f'./data/train_data/{img_id}.jpg'
                    cv2.imwrite(filename, img_cut) 
                    img_id +=1
                    dic = {
                        img_id:data['label']
                    }
                    
                    
                    try:
                        with open("train_label.json", "r+") as old_file:
                            content = json.load(old_file)
                            content.update(dic)
                        with open("train_label.json", "w+") as new_file:
                            json.dump(content,new_file)
                    except:
                        with open("train_label.json", "w+") as new_file:
                            json.dump(dic,new_file)  

for i in conf2:
    path = f"./rgb/test/{i['path'].split('/')[-1]}"
    img = cv2.imread(path,1)
    for data in i['boxes']:
        if data['occluded'] == False:
            x_max=min(data['x_max'],1279)
            x_min=max(data['x_min'],0)
            y_max=min(data['y_max'],719)
            y_min=max(data['y_min'],0)
            x_min,x_max,y_min,y_max= image_box(x_min,x_max,y_min,y_max,width_expanson,height_expension)
            
            if x_max-x_min >= threhold or y_max-y_min >= threhold:
                
                img_cut = cv2.resize(img[y_min:y_max,x_min:x_max,:],(64,64),interpolation = cv2.INTER_LINEAR)

                filename = f'./data/train_data/{img_id}.jpg'
                cv2.imwrite(filename, img_cut) 
                dic = {
                    img_id:data['label']
                }
                img_id +=1
                try:
                    with open("train_label.json", "r+") as old_file:
                        content = json.load(old_file)
                        content.update(dic)
                    with open("train_label.json", "w+") as new_file:
                        json.dump(content,new_file)
                except:
                    with open("train_label.json", "w+") as new_file:
                        json.dump(dic,new_file)  

In [19]:
conf2[0]['boxes'][0]['occluded']

False